In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import  Keys 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.common.by import  By 
from selenium.webdriver.support import  expected_conditions as EC 

import pandas as pd
import numpy as np
import time
import datetime

In [4]:
#建立数据库
columns=['时间','发布日期','作者名','作者类型','粉丝数','投稿数','视频名','分区','子分区','视频类型','自制or转载','标签','视频长度','播放量','点赞数','投币数','收藏数','转发数','评论数','弹幕数','有无分集']
channel_zhishi_zhiye=pd.DataFrame(columns=columns)
channel_zhishi_zhiye

,时间,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,...,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集


In [5]:
driver=webdriver.Chrome()
driver.get('https://passport.bilibili.com/login') 

In [6]:
#登录
driver.find_element_by_xpath('//*[@id="login-username"]').send_keys('15717506191')
driver.find_element_by_xpath('//*[@id="login-passwd"]').send_keys('xy000000')
driver.find_element_by_xpath('//*[@id="geetest-wrap"]/div/div[5]/a[1]').click()
#需要手动过验证码

In [5]:
driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').click()  #选择“知识”频道

In [7]:
#获得分区名
channel=driver.find_element_by_xpath('//*[@id="primaryChannelMenu"]/span[7]/div/a/span').text
channel=channel.strip('0123456789+')
channel

'知识'

In [8]:
driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').click()  #选择“职业职场”子频道
sub_channel=driver.find_element_by_xpath('//*[@id="subnav"]/ul/li[6]/a').text
sub_channel

'职业职场'

In [10]:
driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[1]/ul/a[2]/li').click() #让视频按热度排序

In [11]:
print(datetime.datetime.now())

for month in range(1,11): 
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div/span').click()
    period=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).text
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[1]/div[3]/div[2]/div[2]/div[2]/div/div[2]/a[%d]'%month).click()
    WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]/div/div[2]/div[3]/a'))) 
    for page in range(5):    #爬取每个频道1月到10月每个月播放量前100的视频

        video_loc=1
        while video_loc<21:

            ### 判断作者类型###
            
            
            driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).click()
            driver.switch_to_window(driver.window_handles[-1])
            
            
            author_web_show='No'
            try:
                i=0
                while author_web_show=='No':
                    driver.refresh()
                    i+=1
                    try:
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="n-fs"]')))  #看作者名是否出现
                        WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[3]')))  #看投稿数是否出现
                        author_web_show='Yes'
                    except:
                        author_web_show='No' 
                        if i>3:  
                            raise Exception  #刷新几次后仍爬不了说明用户已注销

                try:
                    driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m organization-auth"]')
                    author_type='机构认证'

                except:
                    try:
                        driver.find_element_by_xpath('//*[@class="user-auth-subscript avatar-m personal-auth"]')
                        author_type='个人认证'
                    except:
                        author_type='无'

                fans_num=driver.find_element_by_xpath('//*[@id="n-fs"]').text  #获取作者粉丝数

                ### 获取作者投稿数 ###
                driver.find_element_by_xpath('//*[@class="n-num"]').click()
                WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span')))  
                video_num=driver.find_element_by_xpath('//*[@id="page-video"]/div/div[1]/div/div/ul/li[1]/span').text

                driver.close()
                driver.switch_to_window(driver.window_handles[-1])


                view_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[1]/span'%video_loc).text #获取播放量
                real_dan_mu_num=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[2]/span[2]/span'%video_loc).text #获取弹幕数
                author=driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/div[3]/a'%video_loc).text #获取作者名
                driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/ul/li[%d]/div/div[2]/a'%video_loc).click() #进入视频界面  
                driver.switch_to_window(driver.window_handles[-1]) 

                try:
                    WebDriverWait(driver,5).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))  #若出现视频被删除的情况则退出

                    video_web_show='No'
                    while video_web_show=='No':
                        driver.refresh()

                        try:
                            WebDriverWait(driver,60).until(EC.presence_of_element_located((By.XPATH,'//*[@class="bilibili-player-video-time-total"]'))) #以视频长度出现时间判定网页是否加载完
                            video_web_show='Yes'

                            try:      
                                driver.find_element_by_xpath('//*[@id="multi_page"]')   #测试是否有分集视频
                                multi='是'

                            except:
                                multi='否'

                            driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')        
                            driver.execute_script('window.scrollTo(document.body.scrollHeight,0)') #再调回最上面是因为，视频长度的元素必须在屏幕内才可以被提取，否则为空

                            bv=driver.current_url[33:44] # 获取BV号（BV号在网址中）
                            print('开始BV号为%s视频属性的抓取'%bv)
                            video_name=driver.find_element_by_xpath('//*[@id="viewbox_report"]/h1/span').text #获取视频名
                            
                            ### 获取发布日期 ###
                            release_date=driver.find_element_by_xpath('//*[@id="viewbox_report"]/div/span[3]').text
                            
                            ### 获取视频类型 ###
                            try:
                                video_type=driver.find_element_by_xpath('//*[@class="activity"]').text
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type=video_type+'和合作视频'
                                except:
                                    pass
                            except:
                                try:
                                    driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[2]/div[1]/div[1]/span[1]')
                                    video_type='合作视频'
                                except:
                                    video_type='普通视频'


                            ### 获取点赞数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text=='点赞':
                                like_num=0
                            else:
                                like_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[1]').text 

                            ### 获取投币数 ###
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text=='投币':
                                coin_num=0
                            else:
                                coin_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').text

                            ### 获取收藏数 ###             
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text=='收藏':
                                collect_num=0
                            else:
                                collect_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[3]').text

                            ### 获取转发数 ###       
                            if driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text=='分享':
                                forward_num=0
                            else:
                                forward_num=driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[4]').text

                            ### 获取视频长度 ###

                            video_length=driver.find_element_by_xpath('//*[@class="bilibili-player-video-time-total"]').text  

                            ### 判断自制还是转载 ###
                            driver.find_element_by_xpath('//*[@id="arc_toolbar_report"]/div[1]/span[2]').click()
                            try:
                                driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/div[2]/div[2]').text
                                self_made='自制'
                            except:
                                self_made='转载'
                            driver.find_element_by_xpath('//*[@id="app"]/div[2]/div[3]/div/div/i').click()

                            ### 获取标签 ###
                            k=1
                            labels=np.array
                            while True:
                                try:
                                    labels=np.append(labels,driver.find_element_by_xpath('//*[@id="v_tag"]/ul/li[%d]'%k).text)
                                    k+=1
                                except: 
                                    break
                            labels=labels[1:]

                            ### 获取评论数 ###
                            driver.execute_script('window.scrollTo(0, 700)')        
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[2]'))) #保证"评论"二字能加载出来                                                   
                            WebDriverWait(driver,20).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div')))
                            if driver.find_element_by_xpath('//*[@id="comment"]/div/div[2]/div/div[4]/div').text=='没有更多信息':    #应对评论区被关闭的情况
                                comment_num='NaN'
                            else:
                                while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                    try:
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[2]/div/div[4]/div[1]')))
                                        while driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text=='':
                                            time.sleep(0.1) #很奇怪，评论数可以定位的一瞬间并不能爬取，会返回空
                                        comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text     
                                    except:
                                        driver.refresh()    #有的时候“评论”二字出现，评论数不出现，页面好像卡住了
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="viewbox_report"]/h1/span')))
                                        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
                                        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="comment"]/div/div[1]/span[1]')))

                                comment_num=driver.find_element_by_xpath('//*[@id="comment"]/div/div[1]/span[1]').text

                            channel_zhishi_zhiye.loc[bv]=[period,release_date,author,author_type,fans_num,video_num,video_name,channel,sub_channel,video_type,self_made,labels,video_length,view_num,like_num,coin_num,collect_num,forward_num,comment_num,real_dan_mu_num,multi]

                            print('BV号为%s的视频属性抓取结束'%bv)

                            driver.close()
                            driver.switch_to_window(driver.window_handles[-1])

                        except:
                            video_web_show='No' 
                except:
                    driver.close()
                    driver.switch_to_window(driver.window_handles[-1])
            except:
                driver.close()
                driver.switch_to_window(driver.window_handles[-1])
            
            video_loc+=1


        driver.find_element_by_xpath('//*[@class="nav-btn iconfont icon-arrowdown3"]').click() #点击下一页
        WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/ul/li[1]')))  #点击下一页后等页面刷新
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[2]').click()  #回到第1页
    WebDriverWait(driver,100).until(EC.presence_of_element_located((By.XPATH,'//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]')))
    driver.find_element_by_xpath('//*[@id="videolist_box"]/div[2]/div[2]/ul/li[1]').click()  #发现很多月份漏爬了约20个，而且恰好是第一页，所以再点一次第一页
        
        
print(datetime.datetime.now()) 

2020-12-21 07:49:29.505926


<ipython-input-11-18eaf7cc69e4>:17: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-18eaf7cc69e4>:54: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])
<ipython-input-11-18eaf7cc69e4>:61: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1uJ41187xc视频属性的抓取
BV号为1uJ41187xc的视频属性抓取结束


<ipython-input-11-18eaf7cc69e4>:181: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1NJ41157xv视频属性的抓取
BV号为1NJ41157xv的视频属性抓取结束
开始BV号为1uJ411H7w4视频属性的抓取
BV号为1uJ411H7w4的视频属性抓取结束
开始BV号为1t7411B7Po视频属性的抓取
BV号为1t7411B7Po的视频属性抓取结束
开始BV号为1yJ411E7Vp视频属性的抓取
BV号为1yJ411E7Vp的视频属性抓取结束
开始BV号为1tJ411p7xK视频属性的抓取
BV号为1tJ411p7xK的视频属性抓取结束
开始BV号为1RJ411n75j视频属性的抓取
BV号为1RJ411n75j的视频属性抓取结束
开始BV号为1GJ41177Re视频属性的抓取
BV号为1GJ41177Re的视频属性抓取结束
开始BV号为1dJ41157U7视频属性的抓取
BV号为1dJ41157U7的视频属性抓取结束
开始BV号为1KJ41177rv视频属性的抓取
BV号为1KJ41177rv的视频属性抓取结束
开始BV号为1NJ41187to视频属性的抓取
BV号为1NJ41187to的视频属性抓取结束
开始BV号为1JJ411L7k1视频属性的抓取
BV号为1JJ411L7k1的视频属性抓取结束
开始BV号为1LJ41187P1视频属性的抓取
BV号为1LJ41187P1的视频属性抓取结束
开始BV号为1SJ411p7gY视频属性的抓取
BV号为1SJ411p7gY的视频属性抓取结束
开始BV号为1xJ411V7TR视频属性的抓取
BV号为1xJ411V7TR的视频属性抓取结束
开始BV号为1C7411z72s视频属性的抓取
BV号为1C7411z72s的视频属性抓取结束
开始BV号为1E7411q7rU视频属性的抓取
BV号为1E7411q7rU的视频属性抓取结束
开始BV号为1pJ411n7pQ视频属性的抓取
BV号为1pJ411n7pQ的视频属性抓取结束
开始BV号为17J411n7k8视频属性的抓取
BV号为17J411n7k8的视频属性抓取结束
开始BV号为1a7411q7p8视频属性的抓取
BV号为1a7411q7p8的视频属性抓取结束
开始BV号为1rJ411J7ZB视频属性的抓取
BV号为1rJ411J7ZB的视频属性抓取结束
开始BV号为1wJ411J7bm视频属性的抓取
BV号为1wJ411J7bm的视

...

BV号为1qD4y1D7SR的视频属性抓取结束
开始BV号为1Fz4y1X7GG视频属性的抓取
BV号为1Fz4y1X7GG的视频属性抓取结束
开始BV号为1v5411W7RD视频属性的抓取
BV号为1v5411W7RD的视频属性抓取结束
开始BV号为1R5411W7zd视频属性的抓取
BV号为1R5411W7zd的视频属性抓取结束
开始BV号为15V41167xn视频属性的抓取
BV号为15V41167xn的视频属性抓取结束
开始BV号为16k4y1q7fY视频属性的抓取
BV号为16k4y1q7fY的视频属性抓取结束
开始BV号为1oK411H7kY视频属性的抓取
BV号为1oK411H7kY的视频属性抓取结束
开始BV号为1dv411z7cy视频属性的抓取
BV号为1dv411z7cy的视频属性抓取结束
开始BV号为1PA411i7c4视频属性的抓取
BV号为1PA411i7c4的视频属性抓取结束
开始BV号为1rf4y127HP视频属性的抓取
BV号为1rf4y127HP的视频属性抓取结束
开始BV号为1np4y1D7do视频属性的抓取
BV号为1np4y1D7do的视频属性抓取结束
开始BV号为1cK411W7QZ视频属性的抓取
BV号为1cK411W7QZ的视频属性抓取结束
开始BV号为14t4y197os视频属性的抓取
BV号为14t4y197os的视频属性抓取结束
开始BV号为1aZ4y1W71s视频属性的抓取
BV号为1aZ4y1W71s的视频属性抓取结束
开始BV号为1Wf4y117hJ视频属性的抓取
BV号为1Wf4y117hJ的视频属性抓取结束
开始BV号为1654y1B7Zi视频属性的抓取
BV号为1654y1B7Zi的视频属性抓取结束
开始BV号为1GV411678x视频属性的抓取
BV号为1GV411678x的视频属性抓取结束
开始BV号为1AC4y1Y7Pr视频属性的抓取
BV号为1AC4y1Y7Pr的视频属性抓取结束
开始BV号为1Qz4y1Q7dh视频属性的抓取
BV号为1Qz4y1Q7dh的视频属性抓取结束
开始BV号为1yi4y1s7NF视频属性的抓取
BV号为1yi4y1s7NF的视频属性抓取结束
开始BV号为11T4y1E72N视频属性的抓取
BV号为11T4y1E72N的视频属性抓取结束
开始BV号为1Jg4y1q7HD

<ipython-input-11-18eaf7cc69e4>:187: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


开始BV号为1np4y1q7XE视频属性的抓取
BV号为1np4y1q7XE的视频属性抓取结束
开始BV号为15T4y1j7HE视频属性的抓取
BV号为15T4y1j7HE的视频属性抓取结束
开始BV号为1cD4y1D7Hm视频属性的抓取
BV号为1cD4y1D7Hm的视频属性抓取结束
开始BV号为1di4y137AR视频属性的抓取
BV号为1di4y137AR的视频属性抓取结束
开始BV号为1xC4y1h7Tc视频属性的抓取
BV号为1xC4y1h7Tc的视频属性抓取结束
开始BV号为1vK411H7jo视频属性的抓取
BV号为1vK411H7jo的视频属性抓取结束
开始BV号为1dZ4y1H7nH视频属性的抓取
BV号为1dZ4y1H7nH的视频属性抓取结束
开始BV号为1sz4y1Q75W视频属性的抓取
BV号为1sz4y1Q75W的视频属性抓取结束
开始BV号为16p4y1i76w视频属性的抓取
BV号为16p4y1i76w的视频属性抓取结束
开始BV号为1GV41167ee视频属性的抓取
BV号为1GV41167ee的视频属性抓取结束
开始BV号为1Mt4y1X7R7视频属性的抓取
BV号为1Mt4y1X7R7的视频属性抓取结束
开始BV号为1fz4y1D7sy视频属性的抓取
BV号为1fz4y1D7sy的视频属性抓取结束
开始BV号为1Mp4y1S759视频属性的抓取
BV号为1Mp4y1S759的视频属性抓取结束
开始BV号为1Tv411q7gx视频属性的抓取
BV号为1Tv411q7gx的视频属性抓取结束
开始BV号为1Ht4y197AJ视频属性的抓取
BV号为1Ht4y197AJ的视频属性抓取结束
开始BV号为1zC4y1h71H视频属性的抓取
BV号为1zC4y1h71H的视频属性抓取结束
开始BV号为1RT4y177u4视频属性的抓取
BV号为1RT4y177u4的视频属性抓取结束
开始BV号为1Ai4y1V7PX视频属性的抓取
BV号为1Ai4y1V7PX的视频属性抓取结束
开始BV号为1ez4y1X7vp视频属性的抓取
BV号为1ez4y1X7vp的视频属性抓取结束
开始BV号为1HZ4y1u7YQ视频属性的抓取
BV号为1HZ4y1u7YQ的视频属性抓取结束
开始BV号为1Di4y137hX视频属性的抓取
BV号为1Di4y137hX的视

BV号为1Xf4y1Q7Fp的视频属性抓取结束
开始BV号为1Xk4y117AR视频属性的抓取
BV号为1Xk4y117AR的视频属性抓取结束
开始BV号为1ST4y1579h视频属性的抓取
BV号为1ST4y1579h的视频属性抓取结束
开始BV号为1dT4y1j7Xg视频属性的抓取
BV号为1dT4y1j7Xg的视频属性抓取结束
开始BV号为1ba4y1E75k视频属性的抓取
BV号为1ba4y1E75k的视频属性抓取结束
开始BV号为15K411M7Yk视频属性的抓取
BV号为15K411M7Yk的视频属性抓取结束
开始BV号为1g64y1F7Fk视频属性的抓取
BV号为1g64y1F7Fk的视频属性抓取结束
开始BV号为1dA411Y7WJ视频属性的抓取
BV号为1dA411Y7WJ的视频属性抓取结束
开始BV号为1TA411e7jV视频属性的抓取
BV号为1TA411e7jV的视频属性抓取结束
开始BV号为18v411i7fo视频属性的抓取
BV号为18v411i7fo的视频属性抓取结束
开始BV号为1jt4y1v7kn视频属性的抓取
BV号为1jt4y1v7kn的视频属性抓取结束
开始BV号为1Xy4y1C7hJ视频属性的抓取
BV号为1Xy4y1C7hJ的视频属性抓取结束
开始BV号为19K4y1a7Hg视频属性的抓取
BV号为19K4y1a7Hg的视频属性抓取结束
开始BV号为1gD4y1o7q1视频属性的抓取
BV号为1gD4y1o7q1的视频属性抓取结束
开始BV号为1tf4y1D75N视频属性的抓取
BV号为1tf4y1D75N的视频属性抓取结束
开始BV号为1hy4y1y7dv视频属性的抓取
BV号为1hy4y1y7dv的视频属性抓取结束
开始BV号为1Up4y1Y7kr视频属性的抓取
BV号为1Up4y1Y7kr的视频属性抓取结束
开始BV号为11a4y1j7kq视频属性的抓取
BV号为11a4y1j7kq的视频属性抓取结束
开始BV号为14T4y1A7QJ视频属性的抓取
BV号为14T4y1A7QJ的视频属性抓取结束
开始BV号为1V541187z5视频属性的抓取
BV号为1V541187z5的视频属性抓取结束
开始BV号为1Za4y1j7rH视频属性的抓取
BV号为1Za4y1j7rH的视频属性抓取结束
开始BV号为1oh41197M4

BV号为1c54y127HT的视频属性抓取结束
开始BV号为1hT4y1c7Ws视频属性的抓取
BV号为1hT4y1c7Ws的视频属性抓取结束
开始BV号为1zz4y1o7oN视频属性的抓取
BV号为1zz4y1o7oN的视频属性抓取结束
开始BV号为14k4y1C7ys视频属性的抓取
BV号为14k4y1C7ys的视频属性抓取结束
开始BV号为1N54y1C7JN视频属性的抓取
BV号为1N54y1C7JN的视频属性抓取结束
开始BV号为1Za411A7aC视频属性的抓取
BV号为1Za411A7aC的视频属性抓取结束
开始BV号为1R54y1178f视频属性的抓取
BV号为1R54y1178f的视频属性抓取结束
开始BV号为16V411y77e视频属性的抓取
BV号为16V411y77e的视频属性抓取结束
开始BV号为1ra4y157wo视频属性的抓取
BV号为1ra4y157wo的视频属性抓取结束
开始BV号为1Ar4y1w7yi视频属性的抓取
BV号为1Ar4y1w7yi的视频属性抓取结束
开始BV号为1aa411A7Aw视频属性的抓取
BV号为1aa411A7Aw的视频属性抓取结束
开始BV号为1qt4y1q7s9视频属性的抓取
BV号为1qt4y1q7s9的视频属性抓取结束
开始BV号为1Zt4y1Y7fm视频属性的抓取
BV号为1Zt4y1Y7fm的视频属性抓取结束
开始BV号为1Zf4y1B78N视频属性的抓取
BV号为1Zf4y1B78N的视频属性抓取结束
开始BV号为1Zr4y1w74X视频属性的抓取
BV号为1Zr4y1w74X的视频属性抓取结束
开始BV号为1sy4y1C7za视频属性的抓取
BV号为1sy4y1C7za的视频属性抓取结束
开始BV号为1Wy4y1r7V8视频属性的抓取
BV号为1Wy4y1r7V8的视频属性抓取结束
开始BV号为1Ki4y177ob视频属性的抓取
BV号为1Ki4y177ob的视频属性抓取结束
开始BV号为1Ua4y1L7vM视频属性的抓取
BV号为1Ua4y1L7vM的视频属性抓取结束
开始BV号为1854y1C7DU视频属性的抓取
BV号为1854y1C7DU的视频属性抓取结束
开始BV号为1C54y1y7Kq视频属性的抓取
BV号为1C54y1y7Kq的视频属性抓取结束
开始BV号为1Af4y1z79h

In [29]:
driver.switch_to_window(driver.window_handles[-1])

<ipython-input-29-dd0a1399d54d>:1: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[-1])


In [12]:
channel_zhishi_zhiye.groupby('时间').agg('count')

,发布日期,作者名,作者类型,粉丝数,投稿数,视频名,分区,子分区,视频类型,自制or转载,标签,视频长度,播放量,点赞数,投币数,收藏数,转发数,评论数,弹幕数,有无分集
时间,,,,,,,,,,,,,,,,,,,,
10月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
1月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
2月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
3月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
4月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
5月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
6月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100
7月,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99,99
8月,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [13]:
channel_zhishi_zhiye.to_json('channel_zhishi_zhiye.json')